In [1]:
import pandas as pd 
from dash import Dash, html, Input, Output,callback, dcc
import dash 
casos = pd.read_csv('caso_full.csv', sep = ',')
import plotly.express as px
import dash_bootstrap_components as dbc


In [65]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 8 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   city                  object
 1   epidemiological_week  int64 
 2   state                 object
 3   new_confirmed         int64 
 4   new_deaths            int64 
 5   ano                   int32 
 6   mes                   int32 
 7   regiao                object
dtypes: int32(2), int64(3), object(3)
memory usage: 205.8+ MB


In [3]:
casos['data'] = pd.to_datetime(casos['date'])
casos['ano'] = casos['data'].dt.year
casos['mes'] = casos['data'].dt.month

In [4]:
casos = casos[['city', 'epidemiological_week', 'state', 'new_confirmed', 'new_deaths', 'ano','mes']]

In [3]:
casos.head()

NameError: name 'casos' is not defined

In [6]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   city                  object
 1   epidemiological_week  int64 
 2   state                 object
 3   new_confirmed         int64 
 4   new_deaths            int64 
 5   ano                   int32 
 6   mes                   int32 
dtypes: int32(2), int64(3), object(2)
memory usage: 176.4+ MB


In [7]:
estados_regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AM': 'Norte',
    'AP': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MG': 'Sudeste',
    'MS': 'Centro-Oeste',
    'MT': 'Centro-Oeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'PR': 'Sul',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RO': 'Norte',
    'RR': 'Norte',
    'RS': 'Sul',
    'SC': 'Sul',
    'SE': 'Nordeste',
    'SP': 'Sudeste',
    'TO': 'Norte'
}


In [2]:
casos['regiao'] = casos['state'].map(estados_regioes)

NameError: name 'casos' is not defined

In [1]:
casos.head()

NameError: name 'casos' is not defined

In [10]:
total_ano = casos[['state','new_deaths','ano','new_confirmed', 'regiao']].groupby(['state','ano','regiao']).sum().reset_index()
total_mes = casos[['state','mes','new_deaths','new_confirmed','ano','regiao']].groupby(['state','mes','ano','regiao']).sum().reset_index()


In [26]:
maior_regiao = total_ano.groupby('regiao')['new_confirmed'].sum().idxmax()
maior_regiao_valor = total_ano.groupby('regiao')['new_deaths'].sum().max()


In [150]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc  # Certifique-se de ter adicionado o import para o dbc

app = dash.Dash(__name__)

# Supondo que você já tenha os DataFrames total_ano e total_estado carregados.

app.layout = html.Div(style={'backgroundColor': '#06141B'}, className="container", children=[
    # Criando os elementos dos filtros
html.Div([
        html.Div(
            dcc.Dropdown(
                id='filtro_ano',
                options=[{'label': ano, 'value': ano} for ano in total_ano['ano'].unique()],
                value=2022,
                style={'color': '#CCD0CF', 'backgroundColor': '#11212D',  # Usando a paleta para o dropdown
                       'borderColor': '#4A5C6A', 'text-align': 'center', 
                       'padding': '5px', 'height': '38px'}
            ), style={'flex': '1', 'margin-right': '10px'}  # Estilo flex para alinhar e espaçamento
        ),

        html.Div(
            dcc.Dropdown(
                id='filtro_estados',
                options=[{'label': estado, 'value': estado} for estado in total_ano['state'].unique()],
                style={'color': '#CCD0CF', 'backgroundColor': '#11212D',  # Usando a paleta para o dropdown
                       'borderColor': '#4A5C6A', 'text-align': 'center',
                       'padding': '5px', 'height': '38px'}
            ), style={'flex': '1'}  # Estilo flex para alinhar e espaçamento
        ),
    ], style={'display': 'flex', 'padding': '10px'}),  # Flexbox para alinhar os elementos


 dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody(html.H4(id='maior_caso', children='Maior Número de Casos')),
        ], style={'backgroundColor': '#CCD0CF', 'color': '#11212D', 'padding': '5px', 'border-radius': '5px', 'margin': '3px'}), width=3),
        
        dbc.Col(dbc.Card([
            dbc.CardBody(html.H4(id='maior_morte', children='Maior Número de Mortes')),
        ], style={'backgroundColor': '#CCD0CF', 'color': '#11212D', 'padding': '5px', 'border-radius': '5px', 'margin': '3px'}), width=3),        
        dbc.Col(dbc.Card([
            dbc.CardBody(html.H4(id='maior_regiao_casos', children='Maior Região com Número de Casos')),
        ], style={'backgroundColor': '#CCD0CF', 'color': '#11212D', 'padding': '5px', 'border-radius': '5px', 'margin': '3px'}), width=3),
        
        dbc.Col(dbc.Card([
            dbc.CardBody(html.H4(id='maior_regiao_mortes', children='Maior Região com Número de Mortes')),
        ], style={'backgroundColor': '#CCD0CF', 'color': '#11212D', 'padding': '5px', 'border-radius': '5px', 'margin': '3px'}), width=3),
    ], style = {'display': 'flex', 'justify-content': 'space-between', 'margin-bottom':'20px'}),
    
    # Criando os elementos dos gráficos
    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico_calor'),
                width=6, style={'padding': '10px', 'color': '#9BA8AB'}),
        dbc.Col(dcc.Graph(id='grafico_hist'), width=6, style={'padding': '10px'}),
    ], style={'display': 'flex'}),

    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico_barras'), width=6, style={'padding': '10px'}),
        dbc.Col(dcc.Graph(id='grafico_rosca'), width=6, style={'padding': '10px'})
    ], style={'display': 'flex'})

])


In [151]:
# Define os Outputs e Inputs do callback
# Define the Outputs and Inputs for the callback
@app.callback(
    [Output('grafico_calor', 'figure'),   # Output do gráfico de mapa coroplético
     Output('grafico_hist', 'figure'),    # Output do gráfico de histograma
     Output('grafico_barras', 'figure'),  # Output do gráfico de barras
     Output('grafico_rosca', 'figure'),   # Output do gráfico de pizza
     Output('maior_caso', 'children'),    # Output da métrica do maior caso
     Output('maior_morte', 'children'),   # Output da métrica do maior número de mortes
     Output('maior_regiao_casos', 'children'), # Output da métrica da região com mais casos
     Output('maior_regiao_mortes', 'children')], # Output da métrica da região com mais mortes
    [Input('filtro_estados', 'value'),    # Input para o filtro de estados
     Input('filtro_ano', 'value')]        # Input para o filtro de anos
)

# Função que atualiza os gráficos e métricas baseado nos filtros aplicados
# Function that updates the charts and metrics based on the applied filters
def atualiza_charts(estado, ano):
    # Filtra os dados para o estado e ano selecionados
    # Filter data for the selected state and year
    estado_df = total_ano[(total_ano['ano'] == ano) & (total_ano['state'] == estado)] if estado else total_ano[total_ano['ano'] == ano]

    # Cria um mapa coroplético para exibir os casos por estado no Brasil
    # Create a choropleth map to display cases by state in Brazil
    fig_mapa = px.choropleth(estado_df,
                             geojson='https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson',
                             locations='state',
                             featureidkey='properties.sigla',
                             color='new_confirmed',
                             hover_name='state',
                             hover_data=['new_confirmed', 'new_deaths'],
                             color_continuous_scale='Blues',
                             title="Mapa Coroplético dos Casos por Estado no Brasil")

    # Atualiza as propriedades geográficas do mapa
    # Update the geographic properties of the map
    fig_mapa.update_geos(
        scope="south america",  # Limita o escopo ao Brasil / Limit scope to Brazil
        projection_type="mercator",  # Usa projeção Mercator / Use Mercator projection
        center={"lat": -14.2350, "lon": -51.9253},  # Centraliza no Brasil / Center on Brazil
        lataxis_range=[-35, 5],  # Ajusta as latitudes para focar no Brasil / Adjust latitudes to focus on Brazil
        lonaxis_range=[-75, -30],  # Ajusta as longitudes para focar no Brasil / Adjust longitudes to focus on Brazil
        landcolor="#11212D",  # Cor da terra / Land color
        showocean=True,  # Exibe os oceanos / Show oceans
        oceancolor="#253745",  # Cor dos oceanos / Ocean color
        lakecolor="#253745"  # Cor dos lagos / Lake color
    )

    # Atualiza o layout do mapa, incluindo o fundo e cor do texto
    # Update the map layout, including background and text color
    fig_mapa.update_layout(
        paper_bgcolor='#253745',
        font=dict(color='#CCD0CF'),
        plot_bgcolor='#CCD0CF'
    )

    # Atualiza as propriedades do eixo X do mapa
    # Update the X-axis properties of the map
    fig_mapa.update_xaxes(
        title_font=dict(size=18, color='#CCD0CF')
    )

    # Filtra os dados apenas pelo ano selecionado
    # Filter data by the selected year only
    filtro_ano = total_ano[total_ano['ano'] == ano]

    # Cria um histograma para comparar o crescimento de mortes por região
    # Create a histogram to compare death growth by region
    fig_hist = px.histogram(filtro_ano,
                            x='regiao',
                            y='new_deaths',
                            hover_name='new_confirmed',
                            color='state',
                            title='Comparação de crescimento de mortes por região.')

    # Atualiza o layout do histograma, incluindo o fundo e cor do texto
    # Update the histogram layout, including background and text color
    fig_hist.update_layout(
        paper_bgcolor='#253745',
        font=dict(color='#CCD0CF')
    )

    # Cria um gráfico de barras para exibir os índices de mortes entre estados
    # Create a bar chart to display death indices between states
    fig_barras = px.bar(data_frame=estado_df,
                        x='new_deaths',
                        y='state',
                        color='regiao',
                        title='Índices de mortes entre estados',
                        orientation='h')

    # Atualiza o layout do gráfico de barras, incluindo o fundo e cor do texto
    # Update the bar chart layout, including background and text color
    fig_barras.update_layout(
        paper_bgcolor='#253745',
        font=dict(color='#CCD0CF')
    )

    # Cria um gráfico de pizza para exibir a porcentagem de casos confirmados por região
    # Create a pie chart to display the percentage of confirmed cases by region
    fig_pie = px.pie(
        filtro_ano,
        values='new_confirmed',
        names='regiao',
        color_discrete_sequence=px.colors.sequential.RdBu,
        labels={'Região': 'regiao'},
        title='Porcentagem de casos confirmados por região'
    )

    # Atualiza as anotações no gráfico de pizza
    # Update annotations on the pie chart
    fig_pie.update_traces(textinfo='label+percent')

    # Atualiza o layout do gráfico de pizza, incluindo o fundo e cor do texto
    # Update the pie chart layout, including background and text color
    fig_pie.update_layout(
        paper_bgcolor='#253745',
        font=dict(color='#CCD0CF')
    )

    # Calcular métricas de maior caso e maior morte
    # Calculate metrics for the largest case and largest death
    maior_caso = estado_df.groupby('state')['new_confirmed'].sum().idxmax()
    maior_caso_valor = estado_df.groupby('state')['new_confirmed'].sum().max()

    maior_morte = estado_df.groupby('state')['new_deaths'].sum().idxmax()
    maior_morte_valor = estado_df.groupby('state')['new_deaths'].sum().max()

    # Calcular métricas da maior região em casos e mortes
    # Calculate metrics for the region with the most cases and deaths
    maior_regiao_casos = estado_df.groupby('regiao')['new_confirmed'].sum().idxmax()
    maior_regiao_casos_valor = estado_df.groupby('regiao')['new_confirmed'].sum().max()

    maior_regiao_mortes = estado_df.groupby('regiao')['new_deaths'].sum().idxmax()
    maior_regiao_mortes_valor = estado_df.groupby('regiao')['new_deaths'].sum().max()

    # Retorna as figuras dos gráficos e as métricas calculadas
    # Return the figures of the charts and the calculated metrics
    return (
        fig_mapa,
        fig_hist,
        fig_barras,
        fig_pie,
        f'Maior número de casos: {maior_caso} ({maior_caso_valor})',
        f'Maior número de mortes: {maior_morte} ({maior_morte_valor})',
        f'Maior região com número de casos: {maior_regiao_casos} ({maior_regiao_casos_valor})',
        f'Maior região com número de mortes: {maior_regiao_mortes} ({maior_regiao_mortes_valor})'
    )

# Inicializa o servidor do Dash
# Initialize the Dash server
if __name__ == '__main__':
    app.run_server(debug=True)
